<a href="https://colab.research.google.com/github/kleczekr/tolkenizer/blob/master/scraping_avocado_post_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import urllib.request
import requests
from tabulate import tabulate
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [3]:
link_base_wohnen = 'https://www.avocadostore.at/wohnen/wohnen-und-leben?page='
result_list_wohnen = list()
# the limit variable restricts the number of pages the function is going to
# iterate through. Full page scrapping would require this number to be 200,
# but that could take up to several hours.
limit_wohnen = 3

In [4]:
def scrap_page(link_base, limit, result_list, start_page):
  page_no = start_page
  while page_no <= limit:
    link = link_base + str(page_no)
    page = urllib.request.urlopen(link)
    page_no += 1
    soup = BeautifulSoup(page, 'html.parser')
    for element in soup.findAll('a', attrs={'class': 'product-item product-item-content'}):
      product_internal_link = element['href']
      product_link = 'https://www.avocadostore.at'+product_internal_link
      product_soup = BeautifulSoup(requests.get(product_link).content, 'html.parser')
      name = product_soup.find('h1').text
      price = product_soup.find('div', attrs={'class': 'product-price'}).text
      try:
        postage = product_soup.find('p', attrs={'class': 'shipping-cost-general'}).text
      except:
        postage = 0
      try:
        brand = product_soup.find('span', attrs={'itemprop': 'seller'}).text
      except:
        brand = 'Brand not given'
      try:
        metadescription = product_soup.find('meta', attrs={'name': 'description'})['content']
      except:
        metadescription = product_soup.find('meta', attrs={'property': 'og:description'})['content']
      try:
        description = product_soup.find('div', attrs={'class': 'col-product-description'}).text
      except:
        description = 'no description'
      try:
        criteria = product_soup.find('div', attrs={'class': 'col-product-criteria'}).text
      except:
        criteria = 'no criteria given'
      result_list.append([name, product_internal_link, price, postage, brand,
                          metadescription, description, criteria])
      print('processed product {}!\n'.format(name))
    print('{}\nProcessed page {}!\n{}\n'.format('*'*60, page_no-1, '*'*60))

In [5]:
column_names = ['name', 'link', 'price', 'postage', 'brand', 'metadescription',
                'description', 'criteria']

In [6]:
scrap_page(link_base_wohnen, limit_wohnen, result_list_wohnen, start_page = 1)
wohnen_df = pd.DataFrame(result_list_wohnen, columns = column_names)
wohnen_df['category'] = 'wohnen'

processed product 
Kopfkissen Bio Lotus Natural®, Bio-Baumwolle kbA, Füllung Kapokfaser
!

processed product 
Geschirrtuch aus Halbleinen mit Bio-Baumwolle 50 x 70 cm
!

processed product 
Feueranzünder
!

processed product 
Duschseife Milchsamt BIO - mit Kokosmilch
!

processed product 
Bio Schoko-Kürbis Zartbitter
!

processed product 
Saatgutkonfetti - kompostierbares Konfetti das Saatgut 24 heimischer (Wild-)Pflanzenarten enthält
!

processed product 
Spork – Bambus Mini-Besteck für unterwegs
!

processed product 
3er Vorratspack Lemongras Seife – Naturkosmetik zertifiziert - biologisch abbaubar
!

processed product 
Costa Rica Cola
!

processed product 
Rasierpinsel KUNSTHAAR mit Olivenholzgriff (vegan)
!

processed product 
22. Türchen - Bluetooth Box
!

processed product 
Glas-Trinkhalme Glas-Strohhalme Trinkhalm Glas + Bürste 6St. 21,5cm
!

processed product 
Veganes Kaugummi Himbeere Vanille
!

processed product 
6x gerade Metallstrohhalme / plastikfreie Trinkhalme inkl. Reini

In [7]:
wohnen_df.head()

,name,link,price,postage,brand,metadescription,description,criteria,category
0,"\nKopfkissen Bio Lotus Natural®, Bio-Baumwolle...",/products/128594-kopfkissen-bio-lotus-natural-...,"\n\n\n\n\n24,77\n\n€\n\n","\nVersand 5,95 €\n/\nVersandkostenfrei ab 90,0...",\nLotus Design\n,Bio Kopfkissen befüllt mit Kapok; Kapok Schlaf...,"\n\nKopfkissen Bio Lotus Natural®, Bio-Baumwol...",\n\nAvocadostore-Kriterien\n\n\n\n\nRohstoffe ...,wohnen
1,\nGeschirrtuch aus Halbleinen mit Bio-Baumwoll...,/products/201249-geschirrtuch-aus-halbleinen-m...,"\n\n\n\n\n10,23\n\n€\n\n","\nVersand 3,95 €\n/\nVersandkostenfrei ab 50,0...",\nKONTOR 1710\n,Geschirrtuch aus Bio-Baumwolle & Leinen fussel...,\n\nGeschirrtuch aus Halbleinen mit Bio-Baumwo...,\n\nAvocadostore-Kriterien\n\n\n\n\nRohstoffe ...,wohnen
2,\nFeueranzünder\n,/products/41162-feueranzuender-if-you-care-iyc,"\n\n\n\n\n2,90\n\n€\n\n\n(8,89€/100stück)\n\n","\nVersand 4,90 €\n/\nVersandkostenfrei ab 100,...",\npromavis\n,ideal für Holzkohlegrill oder Kachelöfen,\n\nFeueranzünder\n\n\n\n\nFeueranzünder aus F...,\n\nAvocadostore-Kriterien\n\n\n\n\nRecycelt &...,wohnen
3,\nDuschseife Milchsamt BIO - mit Kokosmilch\n,/products/186601-duschseife-milchsamt-bio-mit-...,"\n\n\n\n\n7,95\n\n€\n\n\n(8,37)\n\n","\nVersand 5,90 €\n/\nVersandkostenfrei ab 100,...",\nASAVO\n,"Duschseife Milchsamt - handgemachte, vegane Ko...",\n\nDuschseife Milchsamt BIO - mit Kokosmilch\...,\n\nAvocadostore-Kriterien\n\n\n\n\nRohstoffe ...,wohnen
4,\nBio Schoko-Kürbis Zartbitter\n,/products/49787-bio-schoko-kuerbis-zartbitter-...,"\n\n\n\n\n2,49\n\n€\n\n\n(4,98/100g)\n\n","\nVersand 4,90 €\n/\nVersandkostenfrei ab 100,...",\npromavis\n,Bio Landgarten Bio Schoko-Kürbis Zartbitter Ve...,\n\nBio Schoko-Kürbis Zartbitter\n\n\n\n\nBio ...,\n\nAvocadostore-Kriterien\n\n\n\n\nRohstoffe ...,wohnen


In [8]:
print(tabulate(wohnen_df.head(), headers='keys'))

    name                                                                  link                                                                                                                     price             postage                        brand         metadescription                                                                                                                   description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [9]:
wohnen_df.to_csv('wohnen.csv', index=False)
!cp wohnen.csv "drive/My Drive/deite/avocado/"